# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data ="../output_data/weather_df2.csv"
weather_data2 =pd.read_csv(weather_data)
#drop empty cells, just in case
weather_data2_drop = weather_data2.dropna()
del weather_data2_drop["Unnamed: 0"]
weather_data2_drop.head()

,City,Country,Date,Cloudiness,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Nizhniy Kuranakh,RU,1635003722,97,90,58.8353,125.4869,32.72,5.17
1,Kamaishi,JP,1635003724,3,72,39.2667,141.8833,45.27,5.57
2,Puerto Ayora,EC,1635003725,94,78,-0.7393,-90.3518,69.76,9.15
3,Kavieng,PG,1635003726,66,77,-2.5744,150.7967,82.29,15.84
4,Ribeira Grande,PT,1635003728,20,60,38.5167,-28.7000,72.00,10.36


In [3]:
weather_data2_drop.tail()

,City,Country,Date,Cloudiness,Humidity,Lat,Lng,Max Temp,Wind Speed
550,Garoowe,SO,1635004227,4,30,8.4054,48.4845,86.45,7.70
551,Baruun-Urt,MN,1635004501,99,42,46.6806,113.2792,36.50,8.12
552,Belen,TR,1635004503,0,42,36.4891,36.2233,62.53,5.26
553,Laguna,US,1635004505,1,91,38.4210,-121.4238,57.25,3.00
554,Nabari,JP,1635004507,0,77,34.6167,136.0833,50.76,2.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#configure maps

gmaps.configure(api_key=g_key)

#determine location through lat and long
locations = weather_data2_drop[["Lat", "Lng"]]
humid = weather_data2_drop["Humidity"].astype(float)

In [5]:
#plot Heatmap
fig = gmaps.figure()

#heat layer creation
heat_map = gmaps.heatmap_layer(locations, weights=humid, dissipating=False, max_intensity=100, 
                               point_radius=5)

#add layer
fig.add_layer(heat_map)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#determine "no clouds"
no_clouds = weather_data2_drop[weather_data2_drop["Cloudiness"] < 10]
#determine "perfect temp"
perfect_temp_above_65_deg = no_clouds[no_clouds["Max Temp"] > 70]
perfect_temp_below_100_deg = perfect_temp_above_65_deg[perfect_temp_above_65_deg["Max Temp"] < 100]
#determine "cool wind"
cool_wind = perfect_temp_below_100_deg[perfect_temp_below_100_deg["Wind Speed"] < 10]
#determine "comfortable humidity"
comfortable_humidity = cool_wind[cool_wind["Humidity"] < 100]
great_vacation = comfortable_humidity

#resetting the index
great_vacation = great_vacation.reset_index()
del great_vacation["index"]
great_vacation


,City,Country,Date,Cloudiness,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Taoudenni,ML,1635003847,0,15,22.6783,-3.9836,93.33,3.71
1,Santa Isabel Rio Negro,BR,1635003855,9,46,-0.4139,-65.0192,93.79,4.79
2,Salalah,OM,1635003885,0,78,17.0151,54.0924,78.89,5.75
3,Badarganj,BD,1635003940,0,79,25.6740,89.0549,72.93,1.68
4,Richards Bay,ZA,1635003857,9,76,-28.7830,32.0377,75.79,3.20
5,Polis,CY,1635003991,0,79,35.0367,32.4264,71.42,6.26
6,Valle de Allende,MX,1635003996,0,41,26.9333,-105.4000,71.76,4.21
7,Beni Suef,EG,1635004010,0,36,29.0731,31.0979,77.41,9.24
8,Jalu,LY,1635004100,2,26,29.0331,21.5482,80.56,7.67
9,Ratnagiri,IN,1635004146,6,78,16.9833,73.3000,79.07,7.70


In [7]:
#plot vacation locations based on lat and long

locations = great_vacation[["Lat", "Lng"]]
locations_humidity = great_vacation["Humidity"].astype(float)

#heatmap
locations_heatmap = gmaps.figure()
#heat layer
locations_heatlayer = gmaps.heatmap_layer(locations, weights=locations_humidity, dissipating=False, max_intensity=50,
                                         point_radius=2.5)
#add layer
locations_heatmap.add_layer(locations_heatlayer)

#display
locations_heatmap

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(great_vacation["City"])):

    lat = great_vacation.loc[city]["Lat"]
    lng = great_vacation.loc[city]["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Cannot find nearest hotel")

# Dataframe with nearest hotel
great_vacation["Nearest Hotels"] = hotels
great_vacation

,City,Country,Date,Cloudiness,Humidity,Lat,Lng,Max Temp,Wind Speed,Nearest Hotels
0,Taoudenni,ML,1635003847,0,15,22.6783,-3.9836,93.33,3.71,Cannot find nearest hotel
1,Santa Isabel Rio Negro,BR,1635003855,9,46,-0.4139,-65.0192,93.79,4.79,Hotel GSA
2,Salalah,OM,1635003885,0,78,17.0151,54.0924,78.89,5.75,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
3,Badarganj,BD,1635003940,0,79,25.6740,89.0549,72.93,1.68,Raushon
4,Richards Bay,ZA,1635003857,9,76,-28.7830,32.0377,75.79,3.20,BON Hotel Waterfront Richards Bay
5,Polis,CY,1635003991,0,79,35.0367,32.4264,71.42,6.26,Marion
6,Valle de Allende,MX,1635003996,0,41,26.9333,-105.4000,71.76,4.21,Hotel Meson de San Bartolome
7,Beni Suef,EG,1635004010,0,36,29.0731,31.0979,77.41,9.24,Jewel Inn Beni Suef
8,Jalu,LY,1635004100,2,26,29.0331,21.5482,80.56,7.67,صاحبة الفخامة
9,Ratnagiri,IN,1635004146,6,78,16.9833,73.3000,79.07,7.70,Hotel Landmark


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotels}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in great_vacation.iterrows()]
locations = great_vacation[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = [f"Nearby Hotels:{hotels}" for hotel in hotels])
locations_heatmap.add_layer(markers)

# Display figure
locations_heatmap

Figure(layout=FigureLayout(height='420px'))